In [1]:
using Plots
using HDF5
using Unitful
using Statistics
using Distributions
using Plots.PlotMeasures

### Set up the dates array

In [ ]:
using Dates

IDate = "1982010100"
endDate = "1982080100"
SPEEDY_DATE_FORMAT = "YYYYmmddHH"
dates = collect(DateTime(IDate, SPEEDY_DATE_FORMAT):Dates.Hour(6):DateTime(endDate, SPEEDY_DATE_FORMAT))

### Define Functions

In [ ]:
function read_hdf5_file(filename, timestamp, field, level)
    fh = h5open(filename, "r")
    # println("The following datasets found in file ", filename, ": ", keys(fh))
    x_st_unit = read(fh["station_coordinates"]["lon"]["Unit"])
    y_st_unit = read(fh["station_coordinates"]["lat"]["Unit"])
    x_st = read(fh["station_coordinates"]["lon"]) .* uparse(x_st_unit) .|> u"°"
    y_st = read(fh["station_coordinates"]["lat"]) .* uparse(y_st_unit) .|> u"°"
    return fh, x_st, y_st
end

function read_hdf5_truth(filename, timestamp, field, unit_comp, level)
    th = h5open(filename, "r")
    field_unit = "hPa"
    field_desc = read(th["state"][timestamp][field]["Description"]);
    z_truth = read(th["state"][timestamp][field]) .* uparse(field_unit);
    return z_truth[:,:,level].*unit_comp
end

function find_gauge_location(gauge_id, gauges_lon, gauges_lat, state_lon, state_lat)
    pointx = findmin((abs.(state_lon .- ustrip(gauges_lon[gauge_id]))))[2]
    pointy = findmin((abs.(state_lat .- ustrip(gauges_lat[gauge_id]))))[2]
    return pointx, pointy
end

function loop_over_particles(fh, timestamp, field, unit_comp, pointx, pointy)
    field_unit = read(fh["state_particle_1"][timestamp][field]["Unit"])
    field_desc = read(fh["state_particle_1"][timestamp][field]["Description"])
    psA = Float64[]
    for it = 1:256
        particle = read(fh[string("state_particle_",string(it))][timestamp][field]);
        push!(psA, particle[pointx,pointy,level]*unit_comp)
    end
    return psA
end

function plotting(particles, z_truth, gauge_id, pointx, pointy, state_lon, state_lat, timestamp, date, output_folder)
    distA = Normal(mean(particles),std(particles))
    targetx = z_truth[pointx,pointy]
    skew, kurt = round(skewness(particles), digits = 3), round(kurtosis(particles), digits = 3)
    longitude = round(state_lon[pointx], digits = 2)
    latitude = round(state_lat[pointy], digits = 2)
    histogram(particles, xlabel="Surface Pressure", label="", linecolor = "white", normalize = true, top_margin = 12px, title="Observation Location ($longitude °, $latitude °) \n $date")
    vline!([targetx], seriestype = :scatter, color=:red, label="True Value")
    plot!(x->pdf(distA, x), label="Fitted Gaussian", bottom_margin = 12px)
    plot!(size=(1000,400), dpi=300)
    vline!([mean(particles)], seriestype = :scatter, color=:black, label="Mean Value")

    sub_dir = joinpath(output_folder,timestamp) 

    if isdir(sub_dir) == false
        mkdir(sub_dir)
    end
    
    savefig(joinpath(sub_dir,string(gauge_id)))
    
    # if abs(skew) > 0.5 || abs(kurt) > 1.0
    #     @show gauge_id, timestamp
    # end
end

# Load Filter output file

In [ ]:
timestamp = ["t0000", "t0025", "t0050", "t0075", "t0100", "t0125", "t0150", "t0175", "t0200", "t0225", "t0250"]
field = "ps" # Choose from the fields listed above
unit_comp = 0.01 # Convert Pa to hPa
field_unit = "hPa"
output_folder = ""
level = 1

truth_filename = joinpath(output_folder, "obs.h5")
filename = joinpath(output_folder,"optimal.h5")
plot_folder = joinpath(output_folder, "particles")
#Loop through all the gauges and produce the distributions
fh, x_st, y_st = read_hdf5_file(filename, timestamp[1], field, level)
gauges_lon = ustrip(x_st)
gauges_lat = ustrip(y_st)
state_lon = range(0,180,48)
state_lat = range(0,360,96)
for time in timestamp
    number=parse(Int64,split(time,'t')[2]);
    @show time, dates[number+1]
    fh, x_st, y_st = read_hdf5_file(filename, time, field, level)
    z_truth = read_hdf5_truth(truth_filename, time, field, unit_comp, level)
    for gauge_id = 1:50
        pointx, pointy = find_gauge_location(gauge_id, gauges_lon, gauges_lat, state_lon, state_lat)
        particles = loop_over_particles(fh, time, field, unit_comp, pointx, pointy)
        plotting(particles, z_truth, gauge_id, pointx, pointy, state_lon, state_lat, time, dates[number+1], plot_folder)
    end
end